# Real-time vs Prepared Comparison

Comparing the speed of using real-time vs pre-prepared 

In [6]:
import pandas as pd
import psycopg2

# Set up the PostgreSQL connection
database = 'cafes_manhattan'
user = 'cormacegan'
password = ''
host = 'localhost'
port = '5432'

# Connect to the PostgreSQL server
conn = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Execute a SELECT query to retrieve particular predictions
query = "SELECT * FROM predictions WHERE location_id = 4 AND hour = 1 AND day = 1 AND month = 7 AND week_of_month = 1"
cursor.execute(query)

# Fetch the selected predictions into a pandas DataFrame
selected_predictions = pd.DataFrame(cursor.fetchall(), columns=['location_id', 'hour', 'day', 'month', 'week_of_month', 'normalised_prediction'])

# Close the cursor and connection
cursor.close()
conn.close()

# Display the selected predictions
print(selected_predictions)


   location_id  hour  day  month  week_of_month  normalised_prediction
0            4     1    0      7              2               0.001143


We have seen the presence of duplicates in the db (only a relatively small number 1650 out of 578226 rows)

```sql
SELECT COUNT(*) AS duplicate_count
FROM (
    SELECT location_id, hour, day, month, week_of_month
    FROM predictions
    GROUP BY location_id, hour, day, month, week_of_month
    HAVING COUNT(*) > 1
) AS duplicates;
```

We can remove these duplicates using
```sql
DELETE FROM predictions
WHERE (location_id, hour, day, month, week_of_month) IN (
    SELECT location_id, hour, day, month, week_of_month
    FROM predictions
    GROUP BY location_id, hour, day, month, week_of_month
    HAVING COUNT(*) > 1
);
```

This has resulted in 574926 rows

Re running the query from above

In [7]:
import pandas as pd
import psycopg2

# Set up the PostgreSQL connection
database = 'cafes_manhattan'
user = 'cormacegan'
password = ''
host = 'localhost'
port = '5432'

# Connect to the PostgreSQL server
conn = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Execute a SELECT query to retrieve particular predictions
query = "SELECT * FROM predictions WHERE location_id = 4 AND hour = 1 AND day = 1 AND month = 7 AND week_of_month = 1"
cursor.execute(query)

# Fetch the selected predictions into a pandas DataFrame
selected_predictions = pd.DataFrame(cursor.fetchall(), columns=['location_id', 'hour', 'day', 'month', 'week_of_month', 'normalised_prediction'])

# Close the cursor and connection
cursor.close()
conn.close()

# Display the selected predictions
print(selected_predictions)

   location_id  hour  day  month  week_of_month  normalised_prediction
0            4     1    1      7              1               0.001498


## Data Range

From hour 1 day 1 week 1 month 7

Something of an issue in that sometimes there is  a week 0, sometimes not

Hour 0-23
Day 0-6
Month 7-12, 1-6 presumably

Will have to likely rerun prediction generations

### Time for querying the database

In [8]:
import pandas as pd
import psycopg2
import time

# Set up the PostgreSQL connection
database = 'cafes_manhattan'
user = 'cormacegan'
password = ''
host = 'localhost'
port = '5432'

# Connect to the PostgreSQL server
conn = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

start_time = time.time()

# Execute a SELECT query to retrieve particular predictions
query = "SELECT * FROM predictions WHERE location_id = 4 AND hour = 1 AND day = 1 AND month = 7 AND week_of_month = 1"
cursor.execute(query)

# Fetch the selected predictions into a pandas DataFrame
selected_predictions = pd.DataFrame(cursor.fetchall(), columns=['location_id', 'hour', 'day', 'month', 'week_of_month', 'normalised_prediction'])

# Display the selected predictions
print(selected_predictions)

# Measure the elapsed time
query_time = time.time() - start_time
print(f"Query time: {query_time} seconds")

# Close the cursor and connection
cursor.close()
conn.close()

   location_id  hour  day  month  week_of_month  normalised_prediction
0            4     1    1      7              1               0.001498
Query time: 0.03999781608581543 seconds


### Time for generating the prediction

In [ ]:
# import pandas as pd
# import pyarrow as pa
# import pyarrow.parquet as pq
# import pickle

# # Load the pickle files and store them in a dictionary
# models = {}
# location_ids = [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113,
#                 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153,
#                 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231,
#                 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]



In [9]:
import pandas as pd
import pickle
import time

# Load the pickle files and store them in a dictionary
models = {}
location_id = 4  # Specify the desired location_id

# Load the input DataFrame with the specific input values
df_inputs = pd.DataFrame({'hour': [1], 'day': [1], 'month': [7], 'week_of_month': [1]})

# Start timing the operation
start_time = time.time()

# Generate predictions for the specified location_id, hour, day, month, and week of the month
with open(f'pickle_models/model_{location_id}.pkl', 'rb') as file:
    model = pickle.load(file)
    models[location_id] = model
    print(f"Loaded model for location ID {location_id}")

for _, row in df_inputs.iterrows():
    hour = row['hour']
    day = row['day']
    month = row['month']
    week_of_month = row['week_of_month']

    # Create a feature DataFrame with hour, day, month, and week of the month columns
    features = pd.DataFrame({'hour': [hour], 'day': [day], 'month': [month], 'week_of_month': [week_of_month]},
                            columns=['hour', 'day', 'month', 'week_of_month'])

    # Generate prediction using the corresponding model
    prediction = model.predict(features.values)[0]

    # Print the prediction
    print(f"Prediction for location ID {location_id}: {prediction}")

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print(f"Prediction completed in {elapsed_time} seconds")


Loaded model for location ID 4
Prediction for location ID 4: 41.19
Prediction completed in 2.7367208003997803 seconds


## Multiple db queries

In [1]:
import pandas as pd
import psycopg2
import time

# Set up the PostgreSQL connection
database = 'cafes_manhattan'
user = 'cormacegan'
password = ''
host = 'localhost'
port = '5432'

# Connect to the PostgreSQL server
conn = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# List of location IDs
location_ids = [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113,
                114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153,
                158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231,
                232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]

# Start timing the operation
start_time = time.time()

# Generate predictions for each location ID
for location_id in location_ids:
    # Execute a SELECT query to retrieve particular predictions
    query = f"SELECT * FROM predictions WHERE location_id = {location_id} AND hour = 1 AND day = 1 AND month = 7 AND week_of_month = 1"
    cursor.execute(query)

    # Fetch the selected predictions into a pandas DataFrame
    selected_predictions = pd.DataFrame(cursor.fetchall(), columns=['location_id', 'hour', 'day', 'month', 'week_of_month', 'normalised_prediction'])

    # Print the selected predictions
    print(selected_predictions)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print(f"Operation completed in {elapsed_time} seconds")

# Close the cursor and connection
cursor.close()
conn.close()


   location_id  hour  day  month  week_of_month  normalised_prediction
0            4     1    1      7              1               0.001498
   location_id  hour  day  month  week_of_month  normalised_prediction
0           12     1    1      7              1               0.000038
   location_id  hour  day  month  week_of_month  normalised_prediction
0           13     1    1      7              1               0.001013
   location_id  hour  day  month  week_of_month  normalised_prediction
0           24     1    1      7              1               0.001134
   location_id  hour  day  month  week_of_month  normalised_prediction
0           41     1    1      7              1               0.003625
   location_id  hour  day  month  week_of_month  normalised_prediction
0           42     1    1      7              1               0.002557
   location_id  hour  day  month  week_of_month  normalised_prediction
0           43     1    1      7              1               0.001754
   loc

## Multiple Predictions Generations

In [2]:
import pandas as pd
import pickle
import time

# List of location IDs
location_ids = [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113,
                114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153,
                158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231,
                232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]

# Start timing the operation
start_time = time.time()

# Generate predictions for each location ID
for location_id in location_ids:
    # Load the pickle file for the current location_id
    with open(f'pickle_models/model_{location_id}.pkl', 'rb') as file:
        model = pickle.load(file)
        print(f"Loaded model for location ID {location_id}")

    # Load the input DataFrame with the specific input values
    df_inputs = pd.DataFrame({'hour': [1], 'day': [1], 'month': [7], 'week_of_month': [1]})

    for _, row in df_inputs.iterrows():
        hour = row['hour']
        day = row['day']
        month = row['month']
        week_of_month = row['week_of_month']

        # Create a feature DataFrame with hour, day, month, and week of the month columns
        features = pd.DataFrame({'hour': [hour], 'day': [day], 'month': [month], 'week_of_month': [week_of_month]},
                                columns=['hour', 'day', 'month', 'week_of_month'])

        # Generate prediction using the corresponding model
        prediction = model.predict(features.values)[0]

        # Print the prediction
        print(f"Prediction for location ID {location_id}: {prediction}")

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print(f"Operation completed in {elapsed_time} seconds")


Loaded model for location ID 4
Prediction for location ID 4: 41.19
Loaded model for location ID 12
Prediction for location ID 12: 2.03
Loaded model for location ID 13
Prediction for location ID 13: 28.17
Loaded model for location ID 24
Prediction for location ID 24: 31.43
Loaded model for location ID 41
Prediction for location ID 41: 98.27
Loaded model for location ID 42
Prediction for location ID 42: 69.61
Loaded model for location ID 43
Prediction for location ID 43: 48.06
Loaded model for location ID 45
Prediction for location ID 45: 52.47
Loaded model for location ID 48
Prediction for location ID 48: 186.11
Loaded model for location ID 50
Prediction for location ID 50: 47.3
Loaded model for location ID 68
Prediction for location ID 68: 130.85
Loaded model for location ID 74
Prediction for location ID 74: 105.92
Loaded model for location ID 75
Prediction for location ID 75: 66.75
Loaded model for location ID 79
Prediction for location ID 79: 266.42
Loaded model for location ID 87
Pr

## 24 hour window

In [3]:
import pandas as pd
import psycopg2
import time

# Set up the PostgreSQL connection
database = 'cafes_manhattan'
user = 'cormacegan'
password = ''
host = 'localhost'
port = '5432'

# Connect to the PostgreSQL server
conn = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# List of location IDs
location_ids = [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113,
                114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153,
                158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231,
                232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]

# Start timing the operation
start_time = time.time()

# Generate predictions for each location ID and hour
for location_id in location_ids:
    for hour in range(24):
        # Execute a SELECT query to retrieve particular predictions
        query = f"SELECT * FROM predictions WHERE location_id = {location_id} AND hour = {hour} AND day = 2 AND month = 7 AND week_of_month = 1"
        cursor.execute(query)

        # Fetch the selected predictions into a pandas DataFrame
        selected_predictions = pd.DataFrame(cursor.fetchall(), columns=['location_id', 'hour', 'day', 'month', 'week_of_month', 'normalised_prediction'])

        # Print the selected predictions
        print(selected_predictions)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print(f"Operation completed in {elapsed_time} seconds")

# Close the cursor and connection
cursor.close()
conn.close()


   location_id  hour  day  month  week_of_month  normalised_prediction
0            4     0    2      7              1               0.002323
   location_id  hour  day  month  week_of_month  normalised_prediction
0            4     1    2      7              1               0.001452
   location_id  hour  day  month  week_of_month  normalised_prediction
0            4     2    2      7              1                0.00096
   location_id  hour  day  month  week_of_month  normalised_prediction
0            4     3    2      7              1               0.000441
   location_id  hour  day  month  week_of_month  normalised_prediction
0            4     4    2      7              1               0.000301
   location_id  hour  day  month  week_of_month  normalised_prediction
0            4     5    2      7              1               0.000255
   location_id  hour  day  month  week_of_month  normalised_prediction
0            4     6    2      7              1               0.001507
   loc

24 hours worth of predictions for each location ID takes about 28 seconds

## 24 hour window predictions

In [4]:
import pandas as pd
import pickle
import time

# List of location IDs
location_ids = [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113,
                114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153,
                158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231,
                232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]

# Start timing the operation
start_time = time.time()

# Generate predictions for each location ID and hour
for location_id in location_ids:
    # Load the pickle file for the current location_id
    with open(f'pickle_models/model_{location_id}.pkl', 'rb') as file:
        model = pickle.load(file)
        print(f"Loaded model for location ID {location_id}")

    for hour in range(24):
        # Load the input DataFrame with the specific input values
        df_inputs = pd.DataFrame({'hour': [hour], 'day': [1], 'month': [7], 'week_of_month': [1]})

        for _, row in df_inputs.iterrows():
            hour = row['hour']
            day = row['day']
            month = row['month']
            week_of_month = row['week_of_month']

            # Create a feature DataFrame with hour, day, month, and week of the month columns
            features = pd.DataFrame({'hour': [hour], 'day': [day], 'month': [month], 'week_of_month': [week_of_month]},
                                    columns=['hour', 'day', 'month', 'week_of_month'])

            # Generate prediction using the corresponding model
            prediction = model.predict(features.values)[0]

            # Print the prediction
            print(f"Prediction for location ID {location_id}, hour {hour}: {prediction}")

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print(f"Operation completed in {elapsed_time} seconds")


Loaded model for location ID 4
Prediction for location ID 4, hour 0: 57.76
Prediction for location ID 4, hour 1: 41.19
Prediction for location ID 4, hour 2: 21.94
Prediction for location ID 4, hour 3: 11.18
Prediction for location ID 4, hour 4: 8.94
Prediction for location ID 4, hour 5: 7.8
Prediction for location ID 4, hour 6: 37.9
Prediction for location ID 4, hour 7: 94.73
Prediction for location ID 4, hour 8: 144.55
Prediction for location ID 4, hour 9: 170.15
Prediction for location ID 4, hour 10: 107.98
Prediction for location ID 4, hour 11: 101.74
Prediction for location ID 4, hour 12: 111.04
Prediction for location ID 4, hour 13: 123.43
Prediction for location ID 4, hour 14: 113.02
Prediction for location ID 4, hour 15: 129.83
Prediction for location ID 4, hour 16: 143.75
Prediction for location ID 4, hour 17: 184.13
Prediction for location ID 4, hour 18: 254.82
Prediction for location ID 4, hour 19: 235.37
Prediction for location ID 4, hour 20: 173.15
Prediction for location I

Took 6.2 seconds to generate predictions for 24 hour period for each location ID

## Load in pre-made predictions and use that as part of normalisation

In [5]:
import pandas as pd

# Read the min and max values from the CSV file
min_max_values = pd.read_csv('min_max_values.csv')

# Extract the min and max values
min_value = min_max_values['min_value'].item()
max_value = min_max_values['max_value'].item()

# Print the min and max values
print(f"Min Value: {min_value}")
print(f"Max Value: {max_value}")


Min Value: 1.0
Max Value: 26833.44


In [6]:
import pandas as pd
import pickle
import time

# List of location IDs
location_ids = [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 107, 113,
                114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153,
                158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231,
                232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]

# Start timing the operation
start_time = time.time()

# Generate predictions for each location ID and hour
for location_id in location_ids:
    # Load the pickle file for the current location_id
    with open(f'pickle_models/model_{location_id}.pkl', 'rb') as file:
        model = pickle.load(file)
        print(f"Loaded model for location ID {location_id}")

    for hour in range(24):
        # Load the input DataFrame with the specific input values
        df_inputs = pd.DataFrame({'hour': [hour], 'day': [1], 'month': [7], 'week_of_month': [1]})

        for _, row in df_inputs.iterrows():
            hour = row['hour']
            day = row['day']
            month = row['month']
            week_of_month = row['week_of_month']

            # Create a feature DataFrame with hour, day, month, and week of the month columns
            features = pd.DataFrame({'hour': [hour], 'day': [day], 'month': [month], 'week_of_month': [week_of_month]},
                                    columns=['hour', 'day', 'month', 'week_of_month'])

            # Generate prediction using the corresponding model
            prediction = model.predict(features.values)[0]

            # Normalize the prediction using min and max values
            normalized_prediction = (prediction - min_value) / (max_value - min_value)

            # Print the normalized prediction
            print(f"Normalized Prediction for location ID {location_id}, hour {hour}: {normalized_prediction}")

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print(f"Operation completed in {elapsed_time} seconds")


Loaded model for location ID 4
Normalized Prediction for location ID 4, hour 0: 0.0021153499271777
Normalized Prediction for location ID 4, hour 1: 0.0014978138402620112
Normalized Prediction for location ID 4, hour 2: 0.0007803986517812022
Normalized Prediction for location ID 4, hour 3: 0.00037939151266153955
Normalized Prediction for location ID 4, hour 4: 0.000295910472547409
Normalized Prediction for location ID 4, hour 5: 0.00025342458606075334
Normalized Prediction for location ID 4, hour 6: 0.001375201062594382
Normalized Prediction for location ID 4, hour 7: 0.003493159772275649
Normalized Prediction for location ID 4, hour 8: 0.005349867548385462
Normalized Prediction for location ID 4, hour 9: 0.00630393657826124
Normalized Prediction for location ID 4, hour 10: 0.003986965031879322
Normalized Prediction for location ID 4, hour 11: 0.0037544107058471014
Normalized Prediction for location ID 4, hour 12: 0.004101006095606661
Normalized Prediction for location ID 4, hour 13: 0.

## Comparison Conclusions

|  |  DB | Generate|
|----------|----------|----------|
|  Single  |  0.03999781608581543 seconds   | 2.7367208003997803 seconds |
|  66 Zones Single  |   1.1248869895935059 seconds  | 3.443025827407837 seconds |
|  24 Hours X 66 Zones  |   27.89032793045044 seconds  | 6.619454860687256 seconds |

As we can see, as the scale of the predictions increases, so too does the time. This is not surprising. What is surprising is the extent to which the database slows down. This suggests that we may benefit from real time prediction generation. 

Question for the structure of the heatmap

24 hour window?
Current day / time?
Monthly / Yearly Averages?

Option to toggle between them?

If we wish to use averages then using the premade predictions will be useful. 

Note that the min / max values (used in the normalisation) were computing for a years worth of predictions and are read in via a csv
